# BERT fallacy classification


In [1]:
!pip install datasets
!pip install transformers==4.17

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_macro = f1_score(labels, predictions, average="macro")
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "f1": f1
    }

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 6)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.76ea01b4b85ac16e2cec55c398cba7a943d89ab21dfdd973f6630a152e4b9aed
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["Texto"], padding="max_length", truncation=True, max_length=128)

train_df = pd.read_csv("/content/drive/MyDrive/UPV master/HAIA/train_afc.csv")

train_df_split, val_df_split = train_test_split(train_df, test_size=0.2, stratify=train_df['Etiqueta'], random_state=42)

train_df_split = train_df_split.rename(columns={"Etiqueta": "labels"})
val_df_split = val_df_split.rename(columns={"Etiqueta": "labels"})

train_dataset = Dataset.from_pandas(train_df_split)
val_dataset = Dataset.from_pandas(val_df_split)


tokenized_train = train_dataset.map(tokenize_function)
tokenized_valid = val_dataset.map(tokenize_function)
tokenized_train.set_format("torch")
tokenized_valid.set_format("torch")

Map:   0%|          | 0/982 [00:00<?, ? examples/s]

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

In [17]:
# Congelar capas excepto las del encoder y la capa de clasificación
for name, param in model.named_parameters():
    # Congelar las primeras 8 capas del encoder
    if any(f"bert.encoder.layer.{i}" in name for i in range(10)) or "classifier" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # Solo mantiene el mejor modelo
    load_best_model_at_end=True,  # Carga el mejor modelo después del entrenamiento
    metric_for_best_model="f1",  # Métrica para la selección del mejor modelo
    greater_is_better=True,
    num_train_epochs=20,
    weight_decay=0.1,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",  # Coseno de descenso para la tasa de aprendizaje
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Texto. If __index_level_0__, Texto are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 982
  Num Epochs = 20
  Instantaneous batch size per

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1
1,No log,1.128477,0.609756,0.184710,0.521956
2,No log,1.057940,0.646341,0.238228,0.572384
3,No log,1.129098,0.662602,0.295815,0.604766
4,No log,1.207792,0.626016,0.335054,0.601851
5,0.806000,1.324521,0.650407,0.322897,0.611005
6,0.806000,1.573769,0.658537,0.371622,0.620872
7,0.806000,1.608231,0.626016,0.474575,0.624337
8,0.806000,1.611963,0.638211,0.485731,0.631508
9,0.132100,1.647468,0.650407,0.469734,0.636854
10,0.132100,1.703735,0.670732,0.474436,0.648019


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Texto. If __index_level_0__, Texto are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 246
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-123
Configuration saved in ./results/checkpoint-123/config.json
Model weights saved in ./results/checkpoint-123/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-123/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-123/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-246] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Texto. If __index_level_0__, Texto are not expected 

TrainOutput(global_step=2460, training_loss=0.2149685828666377, metrics={'train_runtime': 652.2823, 'train_samples_per_second': 30.11, 'train_steps_per_second': 3.771, 'total_flos': 1291921678725120.0, 'train_loss': 0.2149685828666377, 'epoch': 20.0})

In [19]:
from datasets import concatenate_datasets

# Concatenar los datasets de entrenamiento y validación
full_train_dataset = concatenate_datasets([tokenized_train, tokenized_valid])

In [21]:

# Asegúrate de que el modelo es el mejor modelo obtenido al final del entrenamiento
model = trainer.model

# Reajustar los parámetros de entrenamiento, si lo deseas
final_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # Solo mantiene el mejor modelo
    load_best_model_at_end=True,  # Carga el mejor modelo después del entrenamiento
    metric_for_best_model="f1",  # Métrica para la selección del mejor modelo
    greater_is_better=True,
    num_train_epochs=5,
    weight_decay=0.1,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",  # Coseno de descenso para la tasa de aprendizaje
)

# Crear un nuevo Trainer usando el conjunto de datos combinado
final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=full_train_dataset,  # Usar el conjunto combinado de train y val
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    eval_dataset = tokenized_valid
)

# Entrenar el modelo con todo el conjunto de datos
final_trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Texto. If __index_level_0__, Texto are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1228
  Num Epochs = 5
  Instantaneous batch size per

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1
1,No log,0.325867,0.926829,0.832974,0.924818
2,No log,0.138585,0.955285,0.891286,0.954146
3,No log,0.055476,0.983740,0.962063,0.983427
4,0.130100,0.037616,0.991870,0.974004,0.991635
5,0.130100,0.039319,0.991870,0.974004,0.991635


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Texto. If __index_level_0__, Texto are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 246
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-154
Configuration saved in ./results/checkpoint-154/config.json
Model weights saved in ./results/checkpoint-154/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-154/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-154/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-1722] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, Texto. If __index_level_0__, Texto are not expected

TrainOutput(global_step=770, training_loss=0.10414158461929916, metrics={'train_runtime': 186.3033, 'train_samples_per_second': 32.957, 'train_steps_per_second': 4.133, 'total_flos': 403889974917120.0, 'train_loss': 0.10414158461929916, 'epoch': 5.0})

In [22]:
# Paso 1: Cargar el conjunto de test desde un archivo CSV
test_df = pd.read_csv("/content/drive/MyDrive/UPV master/HAIA/test_afc.csv")

# Paso 2: Renombrar las columnas
test_df = test_df.rename(columns={"Etiqueta": "labels"})  # Si la columna en test también se llama "Etiqueta"

# Paso 3: Convertir a un Dataset de Hugging Face
test_dataset = Dataset.from_pandas(test_df)

# Paso 4: Aplicar la función de tokenización al conjunto de test
tokenized_test = test_dataset.map(tokenize_function)

# Paso 5: Establecer el formato para usarlo con PyTorch
tokenized_test.set_format("torch")

# Ahora puedes usar tokenized_test como entrada para tu modelo.

Map:   0%|          | 0/2160 [00:00<?, ? examples/s]

In [25]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Asegurarse de que el modelo está en el dispositivo correcto
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Paso 1: Preparar los tensores para el DataLoader
# Convertir a tensores y evitar el warning
input_ids = torch.tensor(tokenized_test['input_ids']).clone().detach()
attention_mask = torch.tensor(tokenized_test['attention_mask']).clone().detach()

# Comprobar si token_type_ids están presentes
if 'token_type_ids' in tokenized_test:
    token_type_ids = torch.tensor(tokenized_test['token_type_ids']).clone().detach()
    dataset = TensorDataset(input_ids, attention_mask, token_type_ids)
else:
    dataset = TensorDataset(input_ids, attention_mask)

# Paso 2: Crear DataLoader
dataloader = DataLoader(dataset, batch_size=8)

# Paso 3: Inferencia loop
model.eval()
predictions = []

with torch.no_grad():
    for batch in dataloader:
        # Mover los tensores al dispositivo (GPU o CPU)
        if 'token_type_ids' in batch:
            input_ids_batch, attention_mask_batch, token_type_ids_batch = [t.to(device) for t in batch]
            outputs = model(
                input_ids=input_ids_batch,
                attention_mask=attention_mask_batch,
                token_type_ids=token_type_ids_batch
            )
        else:
            input_ids_batch, attention_mask_batch = [t.to(device) for t in batch]
            outputs = model(
                input_ids=input_ids_batch,
                attention_mask=attention_mask_batch
            )

        # Obtener las predicciones
        logits = outputs.logits
        batch_preds = torch.argmax(logits, dim=-1).cpu().numpy()  # Predecir la clase con mayor probabilidad
        predictions.extend(batch_preds)

# Paso 4: Guardar predicciones en el DataFrame
test_df['predicted_label'] = predictions


<ipython-input-25-e9460c190df2>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokenized_test['input_ids']).clone().detach()
<ipython-input-25-e9460c190df2>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokenized_test['attention_mask']).clone().detach()


In [26]:
test_df

,Texto,labels,predicted_label
0,We got to take a look at what I was left when ...,NaN,0
1,We had an economy that was in free fall.,NaN,0
2,The pandemic was so badly handled.,NaN,0
3,Many people were dying.,NaN,0
4,"All he said was, it's not that serious.",NaN,2
...,...,...,...
2155,She gave a lot of it away to the Taliban.,NaN,0
2156,She gave it to Afghanistan.,NaN,0
2157,What these people have done to our country and...,NaN,0
2158,Many of them are criminals and they're destroy...,NaN,0


In [27]:
# Ver el conteo de cada etiqueta en las predicciones
label_counts = test_df['predicted_label'].value_counts()

# Mostrar los resultados
print(label_counts)

predicted_label
0    1526
2     342
1     201
3      47
5      24
4      20
Name: count, dtype: int64


In [28]:
test_df.to_csv("predicciones.csv", index=False)